#### (1.0) user settings
* `path_project` : project folder containing data in BIDS format
* `path_slicetiming_json`: json file containing SliceTiming field   

In [ ]:
path_project = '/home/kamil/Desktop/Neuroscience/Projects/WD_fasting/'
path_slicetiming_json = '/home/kamil/Desktop/Neuroscience/Projects/WD_fasting/code/WD_fasting_slicetiming.json'

#### (1.1) append missing slice timing field to funcional json files within BIDS structure 

In [41]:
from kbtools import append_json
import os

json_list = []
for path, subdirs, files in os.walk(path_project):
    for name in files:
        if 'bold.json' in name:
            json_list.append(os.path.join(path, name))

for file in json_list:
    append_json(file, path_slicetiming_json)    

#### (1.2) run standard preprocessing using fmriprep

In [29]:
import os

path_project_derivatives = path_project + '/derivatives'
path_project_derivatives_interm = path_project_derivatives + '/intermediate' 

if not os.path.exists(path_project_derivatives_interm):
    !mkdir $path_project_derivatives_interm

#--- look for .bash environmental variable containing fsl license
fsl_test = !echo ${FS_LICENSE:-False}    
  
#--- run preprocessing 
if fsl_test[0] == 'False':
    fs_license_path = input('FSL license not found! Provide full path to fsl license')
    !sudo fmriprep-docker $path_project $path_project_derivatives participant \
    --participant-label wd01 --longitudinal --use-aroma -w $path_project_derivatives_interm \
    --fs-license $fs_license_path
else:
    !sudo fmriprep-docker $path_project $path_project_derivatives participant \
    --participant-label wd01 --longitudinal --use-aroma -w $path_project_derivatives_interm